In [ ]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune
import os
from dotenv import load_dotenv
import pathlib
import re
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
import dsp
import numpy as np
from scipy import sparse
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_distances
import faiss
import json
from dsp.utils.utils import deduplicate
from dspy.datasets import Dataset
from sklearn.model_selection import train_test_split
from typing import Optional, Union
import ast
from bert_score import score
import contractions

In [9]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
import dspy
llm = dspy.HFClientTGI(model="meta-llama/Meta-Llama-3-8B ", port=8090, url="http://127.0.0.1")
dspy.settings.configure(lm=llm)

In [10]:
path_env = pathlib.Path(os.getcwd()).parent.parent / '.env'
load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key

llm = dspy.OpenAI(
    model="gpt-4o", #"gpt-3.5-turbo" , #"gpt-4-0125-preview",  #gpt-4o-2024-05-13, #"gpt-4-1106-preview", # TODO: try turbo-instruct,
    max_tokens=1000)

# Assumes the Weaviate collection has a text key `content`
dspy.settings.configure(lm=llm, trace=[])

## Index

In [40]:
def create_faiss_index(df, text_column, id_column, model_name="all-mpnet-base-v2", index_file="faiss_index.index"):
    """
    Create a FAISS index from a DataFrame containing text data.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    text_column (str): The name of the column containing text data.
    id_column (str): The name of the column containing unique identifiers for the texts.
    model_name (str): The name of the SentenceTransformer model to use for embeddings.
    index_file (str): The file path to save the FAISS index.

    Returns:
    index: The FAISS index object.
    model: The SentenceTransformer model used for embeddings.
    ids: List of document IDs.
    texts: List of document texts.
    """
    texts = df[text_column].tolist()
    ids = df[id_column].tolist()

    model = SentenceTransformer(model_name, device="cuda")

    # Calculate embeddings for the texts
    embeddings = model.encode(texts, show_progress_bar=False)

    # Create a FAISS index
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)  

    # Normalize embeddings to unit length and add to index
    faiss.normalize_L2(embeddings)
    index.add(embeddings)

    # Save the index to a file
    faiss.write_index(index, index_file)

    return index, model, ids, texts

def retrieve_similar_documents(query_text, text_from, model, index, ids, texts, k=5):
    """
    Retrieve the k most similar documents to the query text.

    Parameters:
    query_text (str): The query text.
    model: The SentenceTransformer model used for embeddings.
    index: The FAISS index object.
    ids (list): List of document IDs.
    texts (list): List of document texts.
    k (int): The number of nearest neighbors to retrieve.

    Returns:
    list: A list of dictionaries containing document IDs, distances, and texts of the k most similar documents.
    """
    # Encode the query text
    query_embedding = model.encode([query_text], show_progress_bar=False)
    faiss.normalize_L2(query_embedding)    
    
    # Search the index for the k nearest neighbors
    distances, indices = index.search(query_embedding, k)
        
    # Retrieve the corresponding texts and ids
    
    results = []
    for i in range(k):
        #import pdb; pdb.set_trace()
        if texts[indices[0][i]] != text_from:
            result = {
                "document_id": ids[indices[0][i]],
                "distance": distances[0][i],
                "text": texts[indices[0][i]]
            }
            results.append(result)
        else:
            print("EQUAL")
    
    return results

############
# DATA #####
############
path_orig_en = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_en_tr.parquet")
path_orig_es = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_es_tr.parquet")
path_source = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/passages/translated_stops_filtered_by_al/df_1.parquet")

path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/LDA_FILTERED_AL/rosie_1_20")
path_corpus_en = path_model / "train_data" / "corpus_EN.txt"
path_corpus_es = path_model / "train_data" / "corpus_ES.txt"

persist_directory = (path_model / 'db_contr_mono').as_posix()

raw = pd.read_parquet(path_source)
with path_corpus_en.open("r", encoding="utf-8") as f:
    lines = [line for line in f.readlines()]
corpus_en = [line.rsplit(" 0 ")[1].strip().split() for line in lines]

ids = [line.split(" 0 ")[0] for line in lines]
df_en = pd.DataFrame({"lemmas": [" ".join(doc) for doc in corpus_en]})
df_en["doc_id"] = ids
df_en["len"] = df_en['lemmas'].apply(lambda x: len(x.split()))
df_en["id_top"] = range(len(df_en))
df_en_raw = df_en.merge(raw, how="inner", on="doc_id")[["doc_id", "id_top", "id_preproc", "lemmas_x", "text", "len"]]

# Read thetas 
thetas = sparse.load_npz(path_model.joinpath(f"mallet_output/{'EN'}/thetas.npz")).toarray()
betas = np.load((path_model.joinpath(f"mallet_output/{'EN'}/betas.npy")))
def get_thetas_str(row,thetas):
    return " ".join([f"{id_}|{round(el, 4)}" for id_,el in enumerate(thetas[row]) if el!=0.0])

def get_most_repr_tpc(row,thetas):
    return np.argmax(thetas[row])

# Save thetas in dataframe and "assigned topic"
df_en_raw["thetas"] = df_en_raw.apply(lambda row: get_thetas_str(row['id_top'], thetas), axis=1)
df_en_raw["id_tpc"] = df_en_raw.apply(lambda row: get_most_repr_tpc(row['id_top'], thetas), axis=1)
tpc = 1
df_tpc = df_en_raw[df_en_raw.id_tpc == tpc]

In [41]:
index_en, model_en, ids_en, texts_en = create_faiss_index(df_tpc, text_column='text', id_column='doc_id', index_file='faiss_index_en.index')

/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Create samples with ChatGPT

In [253]:
class GenerateFacts(dspy.Signature):
    """Extract self-contained and fully contextualized facts from the given passage. Each fact should be a complete sentence that makes sense on its own, without relying on pronouns, vague terms, or references to previously mentioned concepts. All key information should be included in each fact."""
    passage = dspy.InputField(desc="The passage may contain one or several facts")
    facts = dspy.OutputField(desc="List of self-contained and fully contextualized claims in the form 'subject + verb + object' without using pronouns or vague references")

class FactsGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_facts = dspy.ChainOfThought(GenerateFacts)

    def forward(self, passage):
        facts = self.generate_facts(passage=passage).facts
    
        return dspy.Prediction(facts=facts)

In [255]:
fact_generator = FactsGenerator()

facts = []
for doc in df_tpc.text.values.tolist()[0:20]:
    facts.append([doc, fact_generator(doc)])

In [256]:
df = pd.DataFrame(facts, columns= ["passage", "facts"])
df["facts"] = df["facts"].apply(lambda x: str([re.sub(r'^\d+\.\s*', '', fact).replace('"',"'") for fact in x.facts.split('\n')]))
df.iloc[4].facts

"['Women who are carriers of the faulty X chromosome have a 1 in 2 chance of passing the faulty X chromosome to a child.', 'Women who are carriers of the faulty X chromosome have a 1 in 2 chance of passing the faulty X chromosome to a child in every pregnancy.', 'If a daughter gets the faulty X chromosome gene, the daughter will likely be a healthy carrier like her mother.', 'If a son gets the faulty X chromosome gene, the son will have X-linked agammaglobulinemia.']"

In [257]:
df

,passage,facts
0,"Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.","['Chorioamnionitis is a condition that occurs during pregnancy.', 'Chorioamnionitis can cause unexplained fever with uterine tenderness.', 'Chorioamnionitis can cause a high white blood cell count.', 'Chorioamnionitis can cause a rapid heart rate in the fetus.', 'Chorioamnionitis can cause a rapid heart rate in the woman.', 'Chorioamnionitis can cause foul-smelling vaginal discharge.']"
1,"Q: I heard that the vitamin K shot might cause cancer. Is this true: A: No. In the early 1990s, a small study in England found an “association” between the vitamin K shot and childhood cancer. An association means that two things are happening at the same time in the same person, but doesn’t tell us whether one causes the other. Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot. If vitamin K was causing cancer, we would expect to see the same association in other groups of children. Scientists looked to see if they could find the same association in other children, but this association between vitamin K and childhood cancer was never found again in any other study.","['In the early 1990s, a small study in England found an association between the vitamin K shot and childhood cancer.', 'An association means that two things are happening at the same time in the same person but does not tell us whether one causes the other.', 'Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot.', 'If vitamin K was causing cancer, scientists would expect to see the same association in other groups of children.', 'Scientists looked to see if they could find the same association between vitamin K and childhood cancer in other children.', 'The association between vitamin K and childhood cancer was never found again in any other study.']"
2,"But what if you know for sure that you never want to be pregnant again, or even pregnant ever? You might be thinking about having your “tubes tied,” or going through a permanent sterilization procedure. But talking to your doctor about sterilization can be tricky! Especially if you’re young, you may worry that your doctor won’t take you seriously or will try to talk you out of it.","['Some people may know for sure that they never want to be pregnant again or ever.', 'Some people might be thinking about having their “tubes tied” or going through a permanent sterilization procedure.', 'Talking to a doctor about sterilization can be tricky.', 'Young people may worry that their doctor won’t take them seriously or will try to talk them out of sterilization.']"
3,Urine test. This is done to look for a substance that may show a certain type of microcephaly.,['A urine test is done to look for a substance that may show a certain type of microcephaly.']
4,"Which children are at risk for X-linked agammaglobulinemia: Women who are carriers have a 1 in 2 chance of passing the faulty X chromosome to a child. This is true for every pregnancy. If a daughter gets the gene, she will likely be a healthy carrier like her mother. If a son gets the gene, he will have X-linked agammaglobulinemia.","['Women who are carriers of the faulty X chromosome have a 1 in 2 chance of passing the faulty X chromosome to a child.', 'Women who are carriers of the faulty X chromosome have a 1 in 2 chance of passing the faulty X chromosome to a child in every pregnancy.', 'If a daughter gets the faulty X chromosome gene, the daughter will likely be a healthy carrier like her mother.', 'If a son gets the faulty X chromosome gene, the son will have X-linked agammaglobulinemia.']"
5,"Your baby can go through periods of increased hunger and fussiness

In [258]:
df.to_csv("facts_gpt4.csv")

## Training Llama

In [396]:
class GenerateFacts(dspy.Signature):
    """
    Extract self-contained and fully contextualized facts from the given passage.    
    """
    
    passage = dspy.InputField(desc="The passage may contain one or several claims")
    facts = dspy.OutputField(desc="List of self-contained and fully contextualized claims in the form 'subject + verb + object' without using pronouns or vague references")
    
class FactsGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_facts = dspy.ChainOfThought(GenerateFacts)

    def process_facts(self, facts):
        nolist = False

        if "Facts:" in facts:
            facts = facts.split("Facts:")[1]
        elif "facts:" in facts:
            facts = facts.split("facts:")[1]

        try:
            facts = contractions.fix(facts)
        except Exception as e:
            print("Could not expand contrad")
            print(e)

        if '’' in facts:
            facts = facts.replace('’', "'")
        if '“' in facts:  # Changed elif to if
            facts = facts.replace('“', "'")
        if '”' in facts:  # Changed elif to if
            facts = facts.replace('”', "'")

        facts = facts.replace('"',"'")

        if "1." in facts:
            try:
                # Process facts, ensuring non-empty lines
                facts = [re.sub(r'^\d+\.\s*', '', fact).replace('"', "'") for fact in facts.split('\n') if fact.strip()]
                return facts
            except Exception as e:
                print("The error is 1")
                print(e)
                print(facts)
                return facts

        
        if facts.startswith("[") and not facts.endswith("]"):
            facts = facts + "]"
        elif not facts.startswith("[") and facts.endswith("]"):
            facts = "[" + facts
        elif not facts.startswith("[") and not facts.endswith("]"):
            nolist = True
            try:
                facts = [el.strip().replace('"',"'") for el in facts.split(".") if len(el) > 1]
                return facts
            except Exception as e:
                print("The error is 2")
                print(e)
                return facts

    
        try:
            facts = [el.strip().replace('"',"'") for el in facts.split(".") if len(el) > 1]#ast.literal_eval(facts)
        except Exception as e:
            print("The error is 3")
            print(e)
            print(facts)
            
        return facts
            
    def forward(self, passage):
        facts = self.generate_facts(passage=passage).facts
        processed_facts = self.process_facts(facts)
        return dspy.Prediction(facts = processed_facts)

In [398]:
fact_generator = FactsGenerator()
doc = df_tpc.text.iloc[12]
fact_generator(doc)

Prediction(
    facts=['Some physical disabilities affect bone density.', 'Cerebral palsy affects bone density.', 'Anorexia affects bone density.', 'Congenital conditions where osteoporosis is a concern affect bone density.', 'Osteoporosis is characterized by low bone density.', 'For people with disabilities that affect bone density, the shot that temporarily decreases bone density is probably not the best option.']
)

In [399]:
class FactsDataset(Dataset):

    def __init__(
        self,
        data_fpath: str,
        dev_size: Optional[float] = 0.2,
        test_size: Optional[float] = 0.2,
        text_key: str = "passage",
        seed: Optional[int] = 11235,
        *args,
        **kwargs
    ) -> None:
        super().__init__(*args, **kwargs)

        self._train = []
        self._dev = []
        self._test = []

        # Read the training data
        train_data = pd.read_csv( pathlib.Path(data_fpath))

        train_data, temp_data = train_test_split(
            train_data, test_size=dev_size + test_size, random_state=seed)
        dev_data, test_data = train_test_split(
            temp_data, test_size=test_size / (dev_size + test_size), random_state=seed)

        self._train = [
            dspy.Example({**row}).with_inputs(text_key) for row in self._convert_to_json(train_data)
        ]
        self._dev = [
            dspy.Example({**row}).with_inputs(text_key) for row in self._convert_to_json(dev_data)
        ]
        self._test = [
            dspy.Example({**row}).with_inputs(text_key) for row in self._convert_to_json(test_data)
        ]

    def _convert_to_json(self, data: pd.DataFrame):
        if data is not None:
            return data.to_dict(orient='records')

dataset = FactsDataset(data_fpath="facts_gpt4.csv", dev_size=0.1)

In [400]:
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
import numpy as np

# Load the SBERT model (you can choose a different pre-trained model if needed)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def combined_score(example, pred, trace=None):
    def sbert_similarity_score(example, pred, trace=None):
        try:
            scores = []
            
            predicted_lst = pred["facts"]
            try:
                gt_lst = ast.literal_eval(example.facts)
            except Exception as e:
                print("Error in parsing ground truth facts: ", e)
                gt_lst = example.facts.split(".")

            min_facts = min(len(predicted_lst), len(gt_lst))

            # Generate embeddings for predicted and ground truth facts
            predicted_embeddings = model.encode(predicted_lst[:min_facts])
            gt_embeddings = model.encode(gt_lst[:min_facts])

            # Calculate cosine similarity for each pair of embeddings
            for pred_emb, gt_emb in zip(predicted_embeddings, gt_embeddings):
                similarity = 1 - cosine(pred_emb, gt_emb)
                scores.append(similarity)

            # Return the average similarity score
            return np.mean(scores)
            
        except Exception as e:
            print("An error occurred: ", e)
            print("predicted_lst: ", predicted_lst)
            print("gt_lst: ", gt_lst)
            return 0.0

    # Now you can use sbert_similarity_score to calculate the similarity
    return sbert_similarity_score(example, pred, trace)

/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [427]:
trainset = dataset._train
devset = dataset._dev
testset = dataset._test


config = dict(max_bootstrapped_demos=4, max_labeled_demos=16, num_candidate_programs=2, max_rounds=2)
teleprompter = BootstrapFewShotWithRandomSearch(metric=combined_score, **config)

compiled_pred = teleprompter.compile(FactsGenerator(), trainset=trainset, valset=devset)

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 2 candidate sets.


  0%|                                                                                                                                                                                                                                                                | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Average Metric: 1.455993663830062 / 2  (72.8): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.93it/s]

Average Metric: 1.455993663830062 / 2  (72.8%)
Score: 72.8 for set: [13]
Scores so far: [79.41, 76.24, 72.68, 76.72, 72.8]
Best score: 79.41
Average of max per entry across top 1 scores: 0.7941108644008636
Average of max per entry across top 2 scores: 0.7941108644008636
Average of max per entry across top 3 scores: 0.7941108644008636
Average of max per entry across top 5 scores: 0.7941108644008636
Average of max per entry across top 8 scores: 0.7941108644008636
Average of max per entry across top 9999 scores: 0.7941108644008636
5 candidate programs found.


In [ ]:
tests = []
for el in testset:
    output = compiled_pred(el.passage)
    tests.append([el.passage, el.facts, output["facts"], combined_score(el, output)])

evaluate = Evaluate(
    devset=devset, metric=combined_score, num_threads=1, display_progress=True)
compiled_score = evaluate(compiled_pred)
uncompiled_score = evaluate(FactsGenerator())

In [404]:
pd.DataFrame(tests)

,0,1,2,3
0,"Q: I heard that the vitamin K shot might cause cancer. Is this true: A: No. In the early 1990s, a small study in England found an “association” between the vitamin K shot and childhood cancer. An association means that two things are happening at the same time in the same person, but doesn’t tell us whether one causes the other. Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot. If vitamin K was causing cancer, we would expect to see the same association in other groups of children. Scientists looked to see if they could find the same association in other children, but this association between vitamin K and childhood cancer was never found again in any other study.","['In the early 1990s, a small study in England found an association between the vitamin K shot and childhood cancer.', 'An association means that two things are happening at the same time in the same person but does not tell us whether one causes the other.', 'Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot.', 'If vitamin K was causing cancer, scientists would expect to see the same association in other groups of children.', 'Scientists looked to see if they could find the same association between vitamin K and childhood cancer in other children.', 'The association between vitamin K and childhood cancer was never found again in any other study.']","[In the early 1990s, a small study in England found an association between the vitamin K shot and childhood cancer., An association means that two things are happening at the same time in the same person but does not indicate causation., Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot., Scientists did not find the same association between vitamin K and childhood cancer in any other study after the initial study in England.]",0.922812
1,"If donor eggs are being used, the same steps are taken. The egg donor will complete ovarian stimulation and egg retrieval. After fertilization takes place, the embryo is transferred to the person who intends to carry the pregnancy (either with or without various fertility medications).","['If donor eggs are being used, the same steps are taken as with non-donor eggs.', 'The egg donor will complete ovarian stimulation.', 'The egg donor will undergo egg retrieval.', 'After fertilization takes place, the embryo is transferred to the person who intends to carry the pregnancy.', 'The embryo transfer to the person who intends to carry the pregnancy can occur with or without various fertility medications.']","[If donor eggs are being used, the same steps are taken as with non-donor eggs., The egg donor will complete ovarian stimulation., The egg donor will complete egg retrieval., After fertilization takes place, the embryo is transferred to the person who intends to carry the pregnancy., The embryo transfer to the person who intends to carry the pregnancy can occur with or without various fertility medications.]",0.995013
2,"Which children are at risk for X-linked agammaglobulinemia: Women who are carriers have a 1 in 2 chance of passing the faulty X chromosome to a child. This is true for every pregnancy. If a daughter gets the gene, she will likely be a healthy carrier like her mother. If a son gets the gene, he will have X-linked agammaglobulinemia.","['Women who are carriers of the faulty X chromosome have a 1 in 2 chance of passing the faulty X chromosome to a child.', 'Women who are carriers of the faulty X chromosome have a 1 in 2 chance of passing the faulty X chromosome to a child in every pregnancy.', 'If a daughter gets the faulty X chromosome gene, the daughter will likely be a healthy carrier like her mother.', 'If a son gets the faulty X chromosome gene, the son will have X-linked agammaglobulinemia.']","[Wo

In [424]:
llm.inspect_history(1)





Extract self-contained and fully contextualized facts from the given passage.

---

Follow the following format.

Passage: The passage may contain one or several claims
Reasoning: Let's think step by step in order to ${produce the facts}. We ...
Facts: List of self-contained and fully contextualized claims in the form 'subject + verb + object' without using pronouns or vague references

---

Passage: Jacqueline Maher, M.D., is a reproductive endocrinology and infertility specialist with a focus on pediatric and adolescent gynecology. She leads the female fertility preservation program at Children's National Hospital to counsel patients in egg freezing or ovarian tissue freezing, and post treatment premature ovarian insufficiency, and hormone replacement/ puberty induction. Her other clinical interests include reproductive endocrine disorders such as premature ovarian insufficiency, polycystic ovary syndrome, precocious puberty, endometriosis and complex contraception; as well as su

In [395]:
compiled_pred("Some physical disabilities also affect bone density, such as cerebral palsy, anorexia, and congenital conditions where osteoporosis (low bone density) is a concern. For people with these disabilities, the shot, which temporarily decreases bone density while you’re using it, probably isn’t the best option.")

Prediction(
    facts=['Some physical disabilities affect bone density, such as cerebral palsy, anorexia, and congenital conditions.', 'Osteoporosis (low bone density) is a concern for people with cerebral palsy, anorexia, and congenital conditions.', 'The shot temporarily decreases bone density while it is being used.', 'The shot probably is not the best option for people with disabilities that affect bone density.']
)

In [386]:
compiled_pred

generate_facts = Predict(<class '__main__.GenerateFacts'>)

In [421]:
compiled_pred.save("compiled_fact.json")

## Question generation

In [101]:
import itertools

In [104]:
facts.iloc[0].facts[0]

'['

In [13]:
questions= pd.read_csv("tr_data/questions_gpt4.csv")
questions

,Unnamed: 0,fact,question
0,0,Chorioamnionitis is a condition that occurs du...,Does chorioamnionitis occur during pregnancy?
1,1,Chorioamnionitis can cause unexplained fever w...,Can chorioamnionitis cause unexplained fever w...
2,2,Chorioamnionitis can cause a high white blood ...,Can chorioamnionitis cause a high white blood ...
3,3,Chorioamnionitis can cause a rapid heart rate ...,Can chorioamnionitis cause a rapid heart rate ...
4,4,Chorioamnionitis can cause a rapid heart rate ...,Can chorioamnionitis cause a rapid heart rate ...
...,...,...,...
77,77,Healthcare providers use ultrasound for many d...,Do healthcare providers use ultrasound for man...
78,78,Healthcare providers use ultrasound to look at...,Do healthcare providers use ultrasound to look...
79,79,Some providers may recommend CA 125 testing co...,Do some providers recommend CA 125 testing com...
80,80,Gestational diabetes usually develops around t...,Does gestational diabetes usually develop arou...


In [6]:
import pandas as pd
import itertools
import ast
facts = pd.read_excel("tr_data/facts_gpt4.xlsx")
facts["facts"] = facts["facts"].apply(lambda x: ast.literal_eval(x))
facts["questions"] = facts["questions"].apply(lambda x: ast.literal_eval(x))

facts

,Unnamed: 0,passage,facts,questions
0,0,Chorioamnionitis: A condition during pregnancy...,[Chorioamnionitis is a condition that occurs d...,"[When does chorioamnionitis occur?, Can chorio..."
1,1,Q: I heard that the vitamin K shot might cause...,"[In the early 1990s, a small study in England ...",[Did a small study in England in the early 199...
2,2,But what if you know for sure that you never w...,[Some people may know for sure that they never...,[Do some people know for sure that they never ...
3,3,Urine test. This is done to look for a substan...,[A urine test is done to look for a substance ...,[Is a urine test done to look for a substance ...
4,4,Which children are at risk for X-linked agamma...,[Women who are carriers of the faulty X chromo...,[What is the change for women who are carriers...
5,5,Your baby can go through periods of increased ...,[Babies can go through periods of increased hu...,[Do babies go through periods of increased hun...
6,7,Note: Positive CRP results also occur during t...,[Positive CRP results occur during the last ha...,"[When positive CRP results occur?, Can the the..."
7,9,I often have a burning sensation in my vagina ...,[A burning sensation in the vagina after a par...,[What I have a burning sensation in the vagina...
8,10,What are Birth Defects: Birth defects are comm...,"[Birth defects are common conditions., Birth d...","[Are birth defects common conditions?, Are bir..."
9,11,Should all women breastfeed? Health care provi...,[Health care providers agree that breastfeedin...,[Do health care providers agree that breastfee...


In [21]:
questions = pd.DataFrame(
    {
        "fact": list(itertools.chain(*facts.facts.values.tolist())),
        "question": list(itertools.chain(*facts.questions.values.tolist())),
    }
)
questions.to_csv("tr_data/questions_gpt4_curated.csv")

In [11]:
import pandas as pd
facts = pd.read_excel("tr_data/facts_gpt4.xlsx")
facts.to_csv("tr_data/facts_gpt4_curated.csv")

In [106]:
facts = pd.read_csv("tr_data/facts_gpt4.csv")
#facts["facts_list"] = df["facts"].apply(lambda x: ast.literal_eval(x))

facts = facts[["passage", "facts", "facts_list", "questions"]]
facts["facts_list"] = facts["facts_list"].apply(lambda x: ast.literal_eval(x))
facts["questions"] = facts["questions"].apply(lambda x: ast.literal_eval(x))

questions_dtset = pd.DataFrame(
    {
        "fact": list(itertools.chain(*facts.facts_list.values.tolist())),
        "question": list(itertools.chain(*facts.questions.values.tolist())),
    }
)

questions_dtset.to_csv("tr_data/questions_gpt4.csv")

In [437]:
class GenerateQuestion(dspy.Signature):
    """Form a close-ended question that directly asks the fact."""
    fact = dspy.InputField()
    question = dspy.OutputField(desc="it asks the fact")

In [440]:
class QAGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_question = dspy.ChainOfThought(GenerateQuestion)
    
    def forward(self, facts):        
        questions = [self.generate_question(fact=fact).question for fact in facts]
        print(f"-- -- Questions: {questions}")
        return questions

In [441]:
qa_generator = QAGenerator()
qa_generator(facts.facts_list.iloc[0])

-- -- Questions: ['Does chorioamnionitis occur during pregnancy?', 'Can chorioamnionitis cause unexplained fever with uterine tenderness?', 'Can chorioamnionitis cause a high white blood cell count?', 'Can chorioamnionitis cause a rapid heart rate in the fetus?', 'Can chorioamnionitis cause a rapid heart rate in the woman?', 'Can chorioamnionitis cause foul-smelling vaginal discharge?']


['Does chorioamnionitis occur during pregnancy?',
 'Can chorioamnionitis cause unexplained fever with uterine tenderness?',
 'Can chorioamnionitis cause a high white blood cell count?',
 'Can chorioamnionitis cause a rapid heart rate in the fetus?',
 'Can chorioamnionitis cause a rapid heart rate in the woman?',
 'Can chorioamnionitis cause foul-smelling vaginal discharge?']

In [443]:
questions = []
for facts_ in facts.facts_list.values.tolist():
    tq = qa_generator(facts_)
    questions.append(tq)

-- -- Questions: ['Does chorioamnionitis occur during pregnancy?', 'Can chorioamnionitis cause unexplained fever with uterine tenderness?', 'Can chorioamnionitis cause a high white blood cell count?', 'Can chorioamnionitis cause a rapid heart rate in the fetus?', 'Can chorioamnionitis cause a rapid heart rate in the woman?', 'Can chorioamnionitis cause foul-smelling vaginal discharge?']
-- -- Questions: ['Did a small study in England in the early 1990s find an association between the vitamin K shot and childhood cancer?', 'Does an association mean that two things are happening at the same time in the same person but does not tell us whether one causes the other?', 'Was it important to figure out whether vitamin K might cause childhood cancer because every newborn is expected to get a vitamin K shot?', 'Do scientists expect to see the same association between vitamin K and cancer in other groups of children?\n\n---', 'Did scientists investigate the association between vitamin K and chil

In [447]:
facts.to_csv("facts_gpt4.csv")

## Questions to Rosie adaptation

In [25]:
import pandas as pd
df_en = pd.read_csv("tr_data/questions_rosie/FullTrialQa7152024.csv")
print(df_en.columns)
df_filtered = df_en.dropna(subset=['question', 'answerPassageText'])
df_filtered[['question', 'answerPassageText']]

Index(['Unnamed: 0', 'answerPassageId', 'createdAt', 'answerPassageText',
       'authorUID', 'question', 'corpusVersion', 'ID', 'Enrollment Date',
       'Name', 'Last Name', 'Email', 'Phone', 'Total Points', 'Money Sent',
       'Compensation Sent On (3 months reward)',
       'TOTAL POINTS      (3 months reward)', 'Total Points.1', 'Money Sent.1',
       'Compensation Sent On (6 months reward)',
       'TOTAL POINTS               (6 months reward)', 'Total Points.2',
       'TOTAL POINTS               (9 months reward)', 'Total Points.3',
       'TOTAL POINTS               (12 months reward)', 'Unnamed: 20'],
      dtype='object')


,question,answerPassageText
0,How long should your kid nurse on each side?,How Long Does Nursing Take: Newborns may nurse...
1,What do I need to do if I have to be rescreene...,For people who do not receive a diagnosis of d...
2,How many ounces of milk should my 2 week old d...,How much milk should I store to meet my baby’s...
3,Do breastfed or formula babies spit up more?,Spitting up and dribbling milk with burps or a...
4,How do you get a breast pump from your insurance,How can I get a breast pump: Under the Patient...
...,...,...
16104,Thanks,Suggested citation for this article: Lengerich...
16105,What if I have a low lying placenta during pre...,What is the difference between placental abrup...
16106,Can you recommend any pelvic floor therapists ...,Do physical therapy: Your health care professi...
16107,Can I give birth vaginally with a placenta pre...,Can I still have a vaginal delivery with place...


In [13]:
from sentence_transformers import SentenceTransformer
from sklearn.svm import OneClassSVM
import numpy as np

# Load pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose different models like 'paraphrase-MiniLM-L6-v2', etc.

def get_bert_embedding(text, tokenizer, model):
    # Tokenize input text and get embeddings
    inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True, padding='max_length')
    with torch.no_grad():
        outputs = model(**inputs)
    # Take the mean of the last hidden state (average of token embeddings)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()
# Sample positive questions (these are the ones from mothers)
positive_questions = list(set(df_filtered[['question', 'answerPassageText']].question.values.tolist()))

positive_embeddings = model.encode(positive_questions)

# Train One-Class SVM on the embeddings of positive questions
ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)
ocsvm.fit(positive_embeddings)

/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/369 [00:00<?, ?it/s]

OneClassSVM(gamma='auto', nu=0.1)

In [20]:
facts["questions_lst"] = facts["questions"].apply(lambda x: ast.literal_eval(x))
facts["facts_list"] = facts["facts_list"].apply(lambda x : ast.literal_eval(x))

In [7]:
import logging
from sentence_transformers import SentenceTransformer
from sklearn.svm import OneClassSVM
import pandas as pd

class QuestionAdapter(object):
    def __init__(
        self,
        path_tr_data="/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/src/qa_system/tr_data/questions_rosie/FullTrialQa7152024.csv",
        logger: logging.Logger = None):
        self._logger = logger or logging.getLogger(__name__)
        self._trf_model = SentenceTransformer('all-MiniLM-L6-v2')
        
        df_positive = pd.read_csv(path_tr_data)
        df_filtered = df_positive.dropna(subset=['question', 'answerPassageText'])
        positive_questions = list(set(df_filtered[['question', 'answerPassageText']].question.values.tolist()))

        self.ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)
        self.ocsvm.fit(self._trf_model.encode(positive_questions))
        
    def predict(self, fact):
        return self.ocsvm.predict(self._trf_model.encode([fact]))[0]

qadat = QuestionAdapter()
qadat.predict("Does chorioamnionitis occur during pregnancy?")

/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1

In [31]:
def keep_rosie_question(questions):
    eq = model.encode(questions)
    predictions = ocsvm.predict(eq)

    keep_questions_ids = [i for i,q in enumerate(questions) if predictions[i]==1]

    return keep_questions_ids

def keep_facts(row):
    facts = row['facts_list']  
    ids = row['keep_questions_ids']  
    return [facts[i] for i in ids] 

def keep_questions(row):
    questions = row['questions_lst']  
    ids = row['keep_questions_ids']  
    return [questions[i] for i in ids] 

facts["keep_questions_ids"] = facts["questions_lst"].apply(keep_rosie_question)
facts["keep_facts"] = facts.apply(keep_facts, axis=1)
facts["keep_questions"] = facts.apply(keep_questions, axis=1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
facts

,Unnamed: 0.1,Unnamed: 0,passage,facts,facts_list,questions,questions_lst,keep_rosie_question,keep_questions_ids,keep_facts,keep_questions
0,0,0,"Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.","['Chorioamnionitis is a condition that occurs during pregnancy.', 'Chorioamnionitis can cause unexplained fever with uterine tenderness.', 'Chorioamnionitis can cause a high white blood cell count.', 'Chorioamnionitis can cause a rapid heart rate in the fetus.', 'Chorioamnionitis can cause a rapid heart rate in the woman.', 'Chorioamnionitis can cause foul-smelling vaginal discharge.']","[Chorioamnionitis is a condition that occurs during pregnancy., Chorioamnionitis can cause unexplained fever with uterine tenderness., Chorioamnionitis can cause a high white blood cell count., Chorioamnionitis can cause a rapid heart rate in the fetus., Chorioamnionitis can cause a rapid heart rate in the woman., Chorioamnionitis can cause foul-smelling vaginal discharge.]","['Does chorioamnionitis occur during pregnancy?', 'Can chorioamnionitis cause unexplained fever with uterine tenderness?', 'Can chorioamnionitis cause a high white blood cell count?', 'Can chorioamnionitis cause a rapid heart rate in the fetus?', 'Can chorioamnionitis cause a rapid heart rate in the woman?', 'Can chorioamnionitis cause foul-smelling vaginal discharge?']","[Does chorioamnionitis occur during pregnancy?, Can chorioamnionitis cause unexplained fever with uterine tenderness?, Can chorioamnionitis cause a high white blood cell count?, Can chorioamnionitis cause a rapid heart rate in the fetus?, Can chorioamnionitis cause a rapid heart rate in the woman?, Can chorioamnionitis cause foul-smelling vaginal discharge?]","[Does chorioamnionitis occur during pregnancy?, Can chorioamnionitis cause unexplained fever with uterine tenderness?, Can chorioamnionitis cause a high white blood cell count?, Can chorioamnionitis cause a rapid heart rate in the fetus?, Can chorioamnionitis cause a rapid heart rate in the woman?, Can chorioamnionitis cause foul-smelling vaginal discharge?]","[0, 1, 2, 3, 4, 5]","[Chorioamnionitis is a condition that occurs during pregnancy., Chorioamnionitis can cause unexplained fever with uterine tenderness., Chorioamnionitis can cause a high white blood cell count., Chorioamnionitis can cause a rapid heart rate in the fetus., Chorioamnionitis can cause a rapid heart rate in the woman., Chorioamnionitis can cause foul-smelling vaginal discharge.]","[Does chorioamnionitis occur during pregnancy?, Can chorioamnionitis cause unexplained fever with uterine tenderness?, Can chorioamnionitis cause a high white blood cell count?, Can chorioamnionitis cause a rapid heart rate in the fetus?, Can chorioamnionitis cause a rapid heart rate in the woman?, Can chorioamnionitis cause foul-smelling vaginal discharge?]"
1,1,1,"Q: I heard that the vitamin K shot might cause cancer. Is this true: A: No. In the early 1990s, a small study in England found an “association” between the vitamin K shot and childhood cancer. An association means that two things are happening at the same time in the same person, but doesn’t tell us whether one causes the other. Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot. If vitamin K was causing cancer, we would expect to see the same association in other groups of children. Scientists looked to see if they could find the same association in other children, but this association between vitamin K and childhood cancer was never found again in any other study.","['In the early 1990s, a small study in England found an association between the vitamin K shot and childhood cancer.', 'An association means that two things are happening at the same time in the same person but 

## Contradictions

In [42]:
class BasicMH(dspy.Module):
    def __init__(self, passages_per_hop=3):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought("context, question -> search_query") for _ in range(2)]
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question, context):
        answer_context = []
        
        for hop in range(2):
            print(f"-- -- Hop {hop}")
            
            search_query = self.generate_query[hop](context=context, question=question).search_query
            
            print(f"-- -- Search query: {search_query}")
            
            passages = retrieve_similar_documents(search_query, context,model_en, index_en, ids_en, texts_en, 5)
            text_passages = [passage["text"] for passage in passages]
            
            print(text_passages)#
            
            answer_context = deduplicate(answer_context + text_passages)
            
            #import pdb; pdb.set_trace()

        return self.generate_answer(context=answer_context, question=question).copy(context=answer_context, text_passage=context)

In [48]:
answer2

Prediction(
    rationale="Reasoning: Let's think step by step in order to produce the answer. We have three pieces of information from the context:\n\n1. The American College of Obstetricians and Gynecologists recommends performing a one-hour blood glucose challenge test to screen for gestational diabetes in low-risk pregnant women between 24 and 28 weeks of pregnancy.\n2. During the 24th to 28th week of pregnancy, a test to check for gestational diabetes, called a blood glucose screening, is typically performed.\n3. Pregnant individuals may be tested for gestational diabetes between the 24th and 28th weeks of pregnancy, with two testing methods available.\n\nAll three pieces of information consistently indicate that the testing for gestational diabetes is recommended and likely to occur between the 24th and 28th weeks of pregnancy.",
    answer='Yes, pregnant individuals will probably be tested for gestational diabetes between 24 and 28 weeks of pregnancy.',
    context=['Gestational

In [51]:
mh = BasicMH()
results = []
for id_row,row in facts.iterrows():

    doc = row["passage"]
    questions = row["keep_questions"]
    answers1 = row["keep_facts"]

    if questions != []:

        for qu, fact in zip(questions, answers1):
            
            print(f"-- -- Question: {qu}")
            answer2 = mh(question=qu, context=doc)
            results.append({"doc": doc, "question": qu, "answer1": fact, "answer2": answer2.answer, "context_answer2": answer2.context})

-- -- Question: Does chorioamnionitis occur during pregnancy?
-- -- Hop 0
-- -- Search query: chorioamnionitis during pregnancy
['What causes chorioamnionitis in pregnancy: A bacterial infection causes chorioamnionitis. The infection can start in your vagina, anus or rectum and spread up to your uterus. Conversely, it can begin in your uterus if the amniotic sac ruptures or tears. The bacteria can infect your placenta or the fetus.', 'Chorioamnionitis can cause complications for the mother and the baby.', 'Chorioamnionitis is a serious condition during pregnancy. While you may not be able to control it, there are some things you can do like recognize signs of the condition and share all of your pregnancy symptoms with your pregnancy care provider. Treatment typically involves antibiotics, but it may also mean your baby will be born early. Discuss your diagnosis with your provider and be sure to ask any questions you have.']
-- -- Hop 1
-- -- Search query: chorioamnionitis during pregna

In [88]:
results_faith_df

,question,answer1,answer2,faith,rationale,context,text_passage
0,Does chorioamnionitis occur during pregnancy?,Chorioamnionitis is a condition that occurs during pregnancy.,"Yes, chorioamnionitis occurs during pregnancy.",True,"Answer 2 is faithful to Answer 1. Both answers state that chorioamnionitis occurs during pregnancy, and there is no contradiction or additional information provided in Answer 2.","[What causes chorioamnionitis in pregnancy: A bacterial infection causes chorioamnionitis. The infection can start in your vagina, anus or rectum and spread up to your uterus. Conversely, it can begin in your uterus if the amniotic sac ruptures or tears. The bacteria can infect your placenta or the fetus., Chorioamnionitis can cause complications for the mother and the baby., Chorioamnionitis is a serious condition during pregnancy. While you may not be able to control it, there are some things you can do like recognize signs of the condition and share all of your pregnancy symptoms with your pregnancy care provider. Treatment typically involves antibiotics, but it may also mean your baby will be born early. Discuss your diagnosis with your provider and be sure to ask any questions you have.]","Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge."
1,Can chorioamnionitis cause unexplained fever with uterine tenderness?,Chorioamnionitis can cause unexplained fever with uterine tenderness.,"Yes, chorioamnionitis can cause unexplained fever with uterine tenderness.",Faithfulness: False,"Both answers are consistent and provide the same information. Answer 2 confirms the information given in Answer 1 without adding any new details or stating that the context does not provide information. Therefore, Answer 2 is faithful to Answer 1.","[Chorioamnionitis - A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge., What causes chorioamnionitis in pregnancy: A bacterial infection causes chorioamnionitis. The infection can start in your vagina, anus or rectum and spread up to your uterus. Conversely, it can begin in your uterus if the amniotic sac ruptures or tears. The bacteria can infect your placenta or the fetus.]","Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge."
2,Can chorioamnionitis cause a high white blood cell count?,Chorioamnionitis can cause a high white blood cell count.,"Yes, chorioamnionitis can cause a high white blood cell count.",Faithfulness: True,"Both answers state that chorioamnionitis can cause a high white blood cell count. Answer 2 does not contradict Answer 1 and does not provide additional information or state that the context does not provide information about the answer. Therefore, Answer 2 is faithful to Answer 1.","[Chorioamnionitis - A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge., Chorioamnionitis is an infection of the placenta and the amniotic fluid.]","Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge."
3,Can chorioamnionitis cause a rapid heart rate in the fetus?,Chorioamnionitis can cause a rapid heart rate in the fetus.,Yes,True,"Answer 2 is faithful to Answer 1. Both answers confirm that chorioamnionitis can cause a rapi

In [4]:
import pandas as pd

df = pd.read_csv("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/src/qa_system/tr_data/questions_gpt4_curated.csv")
df

,Unnamed: 0,fact,question
0,0,Chorioamnionitis is a condition that occurs du...,When does chorioamnionitis occur?
1,1,Chorioamnionitis can cause unexplained fever w...,Can chorioamnionitis cause unexplained fever w...
2,2,Chorioamnionitis can cause a high white blood ...,Can chorioamnionitis cause a high white blood ...
3,3,Chorioamnionitis can cause a rapid heart rate ...,Can chorioamnionitis cause a rapid heart rate ...
4,4,Chorioamnionitis can cause a rapid heart rate ...,Can chorioamnionitis cause a rapid heart rate ...
5,5,Chorioamnionitis can cause foul-smelling vagin...,Can chorioamnionitis cause foul-smelling vagin...
6,6,"In the early 1990s, a small study in England f...",Did a small study in England in the early 1990...
7,7,Figuring out whether vitamin K might cause chi...,Why was it important to figure out whether vit...
8,8,"If vitamin K was causing cancer, scientists wo...",Do scientists expect to see the same associati...
9,9,The association between vitamin K and childhoo...,Was the association between vitamin K and chil...
